## Gateways reached by a Sensor

In [1]:
from datetime import datetime, timedelta
import subprocess
import pandas as pd
from dateutil.parser import parse
from dateutil import tz
from label_map import dev_id_lbls, gtw_lbls

In [2]:
DEVICES = [
    '3148 E 19th, Anc',
    '3414 E 16th, Anc',
    '3424 E 18th, Anc',
    '122 N Bliss',
    '122 N Bliss Unit',
    '1826 Columbine, Anc',
    'Phil ELT-2 3692',
    'Phil LT22222 436E',
    'Phil CO2 26D8',
]

# Days of Data to Show
DAYS = 4

GATEWAY_FILE = '../an-api/lora-data/gateways.tsv'

In [3]:
# Make DateTime objects for time period analyze
tz_ak = tz.gettz('US/Alaska')
start_ts = (datetime.now(tz_ak) - timedelta(days=DAYS)).replace(
    tzinfo=None, minute=0, second=0, microsecond=0)
end_ts = datetime.now(tz_ak).replace(
    tzinfo=None, minute=0, second=0, microsecond=0)
start_ts, end_ts

(datetime.datetime(2021, 4, 26, 16, 0), datetime.datetime(2021, 4, 30, 16, 0))

In [9]:
# Pick a device to use for testing the script
device = DEVICES[3]

df = pd.read_csv(GATEWAY_FILE, 
    sep='\t', 
    parse_dates=['ts', 'ts_hour'],
    index_col='ts',
    low_memory=False)
df = df.loc[str(start_ts):]
df['dev_id'] = df.dev_id.map(dev_id_lbls)
df.query('dev_id == @device', inplace=True)

def gtw_map(gtw_eui):
    return gtw_lbls.get(gtw_eui, gtw_eui)

df['gateway'] = df.gateway.map(gtw_map)
df.head()

,dev_id,ts_day,ts_hour,counter,gateway,snr,rssi,data_rate
ts,,,,,,,,
2021-04-26 16:01:00,122 N Bliss,2021-04-26,2021-04-26 16:00:00,15141,AHFC HQ Outdoor,-12.5,-118,SF10BW125
2021-04-26 16:11:00,122 N Bliss,2021-04-26,2021-04-26 16:00:00,15142,Tyler Outdoor,-13.0,-134,SF10BW125
2021-04-26 17:01:00,122 N Bliss,2021-04-26,2021-04-26 17:00:00,15147,Tyler Outdoor,-11.0,-129,SF10BW125
2021-04-26 17:11:00,122 N Bliss,2021-04-26,2021-04-26 17:00:00,15148,Tyler Outdoor,-11.2,-130,SF10BW125
2021-04-26 18:21:00,122 N Bliss,2021-04-26,2021-04-26 18:00:00,15155,Tyler Outdoor,-10.8,-132,SF10BW125


In [10]:
# Determine the "Any" gateway counts by dropping duplicate readings
df_any = df[['ts_hour', 'counter']].drop_duplicates(subset=['counter'])
df_any_count = df_any.groupby('ts_hour').count()
df_any_count.columns = ['Any' ]
df_any_count

,Any
ts_hour,
2021-04-26 16:00:00,2
2021-04-26 17:00:00,2
2021-04-26 18:00:00,2
2021-04-26 19:00:00,3
2021-04-26 20:00:00,4
...,...
2021-04-30 10:00:00,3
2021-04-30 11:00:00,5
2021-04-30 12:00:00,2


In [11]:
# Determine counts for individual gateways
df_cts = pd.pivot_table(df, index='ts_hour', columns='gateway', values='counter', aggfunc='count')
df_cts

gateway,AHFC HQ Outdoor,Alan Outdoor,Kasilof,Tyler Outdoor
ts_hour,,,,
2021-04-26 16:00:00,1.0,NaN,NaN,1.0
2021-04-26 17:00:00,NaN,NaN,NaN,2.0
2021-04-26 18:00:00,1.0,NaN,NaN,1.0
2021-04-26 19:00:00,2.0,NaN,1.0,2.0
2021-04-26 20:00:00,NaN,3.0,1.0,2.0
...,...,...,...,...
2021-04-30 10:00:00,2.0,2.0,1.0,1.0
2021-04-30 11:00:00,1.0,1.0,NaN,4.0
2021-04-30 12:00:00,NaN,NaN,1.0,1.0


In [12]:
# Combine the two DataFrames, horizontally (combine columns)
df_final = pd.concat([df_any_count, df_cts], axis=1)

# Make a new index that fills in any missing hours
new_ix = pd.date_range(start_ts, end_ts, freq='1H')
df_final = df_final.reindex(new_ix)

# Replace NaN values with zero and then convert values to integers
df_final.fillna(0, inplace=True)
df_final = df_final.astype('int32')

df_final = df_final[:-1]    # drop last hour because likely incomplete

# Convert index into a string so we can change drop the seconds from display
df_final.index = df_final.index.strftime("%Y-%m-%d %H:%M")

df_final

,Any,AHFC HQ Outdoor,Alan Outdoor,Kasilof,Tyler Outdoor
2021-04-26 16:00,2,1,0,0,1
2021-04-26 17:00,2,0,0,0,2
2021-04-26 18:00,2,1,0,0,1
2021-04-26 19:00,3,2,0,1,2
2021-04-26 20:00,4,0,3,1,2
...,...,...,...,...,...
2021-04-30 11:00,5,1,1,0,4
2021-04-30 12:00,2,0,0,1,1
2021-04-30 13:00,1,0,1,0,0
2021-04-30 14:00,1,0,1,1,1


In [13]:
def color_cells(val):
    color_scale = {
        0: '#FF3131',
        1: '#FFFF00',
        2: '#FFD822',
    }    
    color = color_scale.get(val, '#EEEEEE')
    return 'background: %s' % color

s = df_final.style
s.applymap(color_cells)
s.set_properties(**{'width': '70px', 'text-align': 'center'})
styles = [
    dict(selector="td", props=[('padding', "0px")]),
]         
s.set_table_styles(styles)

s

,Any,AHFC HQ Outdoor,Alan Outdoor,Kasilof,Tyler Outdoor
2021-04-26 16:00,2,1,0,0,1
2021-04-26 17:00,2,0,0,0,2
2021-04-26 18:00,2,1,0,0,1
2021-04-26 19:00,3,2,0,1,2
2021-04-26 20:00,4,0,3,1,2
2021-04-26 21:00,2,2,0,0,1
2021-04-26 22:00,2,1,1,1,1
2021-04-26 23:00,4,0,1,0,4
2021-04-27 00:00,4,2,3,3,1
2021-04-27 01:00,4,1,2,4,2
